In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset
from nn import CustomImageDataset, FeedForwardNN, test_model, validate, CNN
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [2]:
input_size = 28 * 28  # Flattened image size (28x28)
num_classes = 94  # Number of classes (modify if needed)
learning_rate = 0.001
num_epochs = 20
batch_size = 64

dataset = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="train")
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataset_test = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
val_dataset = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="validation")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)



In [10]:
# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

Epoch [1/20], Train Loss: 2.7498, Validation Loss: 1.9117, Validation Accuracy: 46.92%
Validation loss improved! Saving model...
Epoch [2/20], Train Loss: 1.6994, Validation Loss: 1.5619, Validation Accuracy: 55.33%
Validation loss improved! Saving model...
Epoch [3/20], Train Loss: 1.4310, Validation Loss: 1.3562, Validation Accuracy: 61.00%
Validation loss improved! Saving model...
Epoch [4/20], Train Loss: 1.2546, Validation Loss: 1.2032, Validation Accuracy: 64.62%
Validation loss improved! Saving model...
Epoch [5/20], Train Loss: 1.1320, Validation Loss: 1.1827, Validation Accuracy: 65.64%
Validation loss improved! Saving model...
Epoch [6/20], Train Loss: 1.0490, Validation Loss: 1.1513, Validation Accuracy: 66.40%
Validation loss improved! Saving model...
Epoch [7/20], Train Loss: 0.9924, Validation Loss: 1.0263, Validation Accuracy: 69.87%
Validation loss improved! Saving model...
Epoch [8/20], Train Loss: 0.9380, Validation Loss: 0.9554, Validation Accuracy: 71.20%
Validation

In [11]:
dataset_test = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
test_model(model, data_loader_test, device, criterion)

Test Loss: 0.7740, Test Accuracy: 76.05%


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

Epoch [1/20], Train Loss: 2.0613, Validation Loss: 1.3183, Validation Accuracy: 62.37%
Validation loss improved! Saving model...
Epoch [2/20], Train Loss: 1.0521, Validation Loss: 0.9128, Validation Accuracy: 72.68%
Validation loss improved! Saving model...
Epoch [3/20], Train Loss: 0.7994, Validation Loss: 0.7949, Validation Accuracy: 76.06%
Validation loss improved! Saving model...
Epoch [4/20], Train Loss: 0.6992, Validation Loss: 0.7041, Validation Accuracy: 78.04%
Validation loss improved! Saving model...
Epoch [5/20], Train Loss: 0.6394, Validation Loss: 0.6568, Validation Accuracy: 79.64%
Validation loss improved! Saving model...
Epoch [6/20], Train Loss: 0.5981, Validation Loss: 0.6380, Validation Accuracy: 79.73%
Validation loss improved! Saving model...
Epoch [7/20], Train Loss: 0.5668, Validation Loss: 0.6233, Validation Accuracy: 80.53%
Validation loss improved! Saving model...
Epoch [8/20], Train Loss: 0.5404, Validation Loss: 0.5938, Validation Accuracy: 80.95%
Validation

In [5]:
test_model(model, data_loader_test, device, criterion)

Test Loss: 0.4974, Test Accuracy: 83.93%
